In [1]:
### SVPINO uses loader.load_and_split() from PyPDFLoader (pypdf)
### https://ollama.com/blog/embedding-models

In [2]:
import os
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.prompts import PromptTemplate
from operator import itemgetter

In [3]:
MODEL = 'llama2'
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

loader = PyPDFLoader('Pregnancy and social welfare payments.pdf')
pages = loader.load_and_split()

In [4]:
# Chaining test
parser = StrOutputParser()
chain = model | parser
print(chain.invoke('Tell me a joke'))


Why don't scientists trust atoms? Because they make up everything! 😂


In [5]:
# PromptTemplate testing
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context='Here is some context', question='Here is a question'))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know"

Context: Here is some context

Question: Here is a question



In [6]:
# Vectorstore
vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

retriever = vectorstore.as_retriever()
for chunk in retriever.invoke("payment"):
    print(chunk)
    break

page_content='4/12/24, 2:46 PM Pregnancy and social welfare payments\nhttps://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 3/3If you are taking part in an employment scheme you have the full range of statutory employment\nrights including maternity leave.\nIf you become pregnant while on an employment scheme (Community Employment, Rural Social\nScheme, Tús and Gateway schemes) you are en\x00tled to take 26 weeks’ maternity leave including an\naddi\x00onal 16 weeks of op\x00onal unpaid leave. If possible, you will be supported to complete your 52\nweeks of paid work on the same or another project when you ﬁnish your maternity leave.\nYou should discuss with your sponsor the payment you qualify for while you are on maternity leave.\nYou may qualify for Maternity Beneﬁt if you have enough PRSI contribu\x00ons. Otherwise, if your\nspouse or partner is ge\x00ng a social welfare payment, you may get a qualiﬁed adult increase

In [13]:
chain = (

    {
        "context": itemgetter('question') | retriever,
        "question": itemgetter('question')
    }
    | prompt
    | model
    | parser
)

questions = [
    "What are the conditions to continue receiving social welfare payments if pregnant?",
    "How should one apply for Maternity Benefit and what are the deadlines?",
    "Under what conditions can a pregnant woman qualify for the Working Family Payment?",
    "What are the requirements for receiving Jobseekers Benefit or Allowance while pregnant?",
    "How does the One-Parent Family Payment change when a baby is born?",
    "What are the rules for switching from Illness Benefit to Maternity Benefit?",
    "Can you participate in employment schemes while pregnant and what rights do you have?"
]

c = 1
for q in questions:
    print(f"Question {c}: {q}")
    print(f"Answer: {chain.invoke({'question': {q}})}")
    print(f"\n")
    c +=1

Question 1: What are the conditions to continue receiving social welfare payments if pregnant?
Answer: The conditions to continue receiving social welfare payments if pregnant are as follows:

1. You must be available for and genuinely seeking work throughout your pregnancy.
2. You must be capable of work unless you have complications during your pregnancy or you are ill.
3. You must be claiming Jobseeker's Beneﬁt or Jobseeker's Allowance when you become pregnant.
4. If you are on maternity leave from work, you may qualify for Maternity Beneﬁt if you have paid enough social insurance (PRSI).
5. If you do not qualify for Maternity Beneﬁt, you may be able to stay on your current payment or transfer to another payment that better suits your needs.
6. If you are pregnant and have no other children, you will not qualify for the Working Family Payment (WFP) unless your baby is born.
7. You must tell your Intreo Centre or local Social Welfare Branch Oﬃce about your pregnancy to ensure that yo

In [19]:
question = "Under what conditions can a pregnant woman qualify for the Working Family Payment? \n"
print(f'{question}')
for s in chain.stream({'question': question}):
    print(s, end="", flush=True)

Under what conditions can a pregnant woman qualify for the Working Family Payment? 

Under the following conditions, a pregnant woman can qualify for the Working Family Payment:

1. She must be on maternity leave from work and have paid enough social insurance (PRSI).
2. She must meet the conditions for the Working Family Payment.
3. If she is pregnant and has no other children, she will not qualify for the Working Family Payment until after her baby is born.